In [2]:
'hcp.recobundles' + f'/sub-{2}/sub-{3}_csd.nii.gz'

'hcp.recobundles/sub-2/sub-3_csd.nii.gz'

In [7]:
from AFQ.api import make_bundle_dict
bundles_waypoint_roi = set(make_bundle_dict().keys())
print(bundles_waypoint_roi)

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/templates 
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/callosum_templates 
{'IFO_R', 'CGC_L', 'FP', 'ATR_L', 'UNC_L', 'FA', 'CST_R', 'HCC_R', 'SLF_L', 'CST_L', 'ILF_R', 'ILF_L', 'IFO_L', 'ATR_R', 'SLF_R', 'HCC_L', 'ARC_R', 'UNC_R', 'CGC_R', 'ARC_L'}


In [6]:
from AFQ.data import read_hcp_atlas_16_bundles
bundles_reco = set(read_hcp_atlas_16_bundles().keys())
print(bundles_reco)

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/hcp_atlas_16_bundles 
{'AF_L', 'F_R', 'CST_L', 'IFOF_R', 'F_L', 'UF_R', 'C_L', 'CST_R', 'UF_L', 'CCMid', 'MCP', 'whole_brain', 'CC_ForcepsMajor', 'CC_ForcepsMinor', 'AF_R', 'C_R', 'IFOF_L'}


In [8]:
print(bundles_waypoint_roi & bundles_reco)

{'CST_L', 'CST_R'}


In [ ]:
# IFOF is IFO
# UNC is UF
# AF is ARC
# CST is CST
# FA and FP maybe related to F_L and F_R

# F-1 score comparing set membership
# dice coefficient between covered voxels (1 or 0 map)
# compare bundle profiles

In [1]:
import configparser
import os.path as op

CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), 'aws', 'credentials')))
CP.sections()
ak = CP.get('hcp', 'AWS_ACCESS_KEY_ID')
sk = CP.get('hcp', 'AWS_SECRET_ACCESS_KEY')

In [10]:
import numpy as np
def clean_fiber_group(streamlines, n_points=100, clean_rounds=5,
                      clean_threshold=3, min_sl=20, stat=np.mean):
    """
    Clean a segmented fiber group based on the Mahalnobis distance of
    each streamline
    Parameters
    ----------
    streamlines : nibabel.Streamlines class instance.
        The streamlines constituting a fiber group.
        If streamlines is None, will use previously given streamlines.
        Default: None.
    clean_rounds : int, optional.
        Number of rounds of cleaning based on the Mahalanobis distance from
        the mean of extracted bundles. Default: 5
    clean_threshold : float, optional.
        Threshold of cleaning based on the Mahalanobis distance (the units are
        standard deviations). Default: 3.
    min_sl : int, optional.
        Number of streamlines in a bundle under which we will
        not bother with cleaning outliers. Default: 20.
    stat : callable, optional.
        The statistic of each node relative to which the Mahalanobis is
        calculated. Default: `np.mean` (but can also use median, etc.)
    Returns
    -------
    A nibabel.Streamlines class instance containing only the streamlines
    that have a Mahalanobis distance smaller than `clean_threshold` from
    the mean of each one of the nodes.
    """

    # We don't even bother if there aren't enough streamlines:
    if len(streamlines) < min_sl:
        return streamlines

    # Resample once up-front:
    fgarray = _resample_bundle(streamlines[:]['sl'], n_points)
    # Keep this around, so you can use it for indexing at the very end:
    idx = np.arange(len(fgarray))
    # This calculates the Mahalanobis for each streamline/node:
    w = gaussian_weights(fgarray, return_mahalnobis=True, stat=stat)
    # We'll only do this for clean_rounds
    rounds_elapsed = 0
    while (np.any(w > clean_threshold)
           and rounds_elapsed < clean_rounds
           and len(streamlines) > min_sl):
        # Select the fibers that have Mahalanobis smaller than the
        # threshold for all their nodes:
        idx_belong = np.where(
            np.all(w < clean_threshold, axis=-1))[0]
        idx = idx[idx_belong.astype(int)]
        # Update by selection:
        fgarray = fgarray[idx_belong.astype(int)]
        # Repeat:
        w = gaussian_weights(fgarray, return_mahalnobis=True)
        rounds_elapsed += 1
    # Select based on the variable that was keeping track of things for us:
    return streamlines[idx]

In [25]:
# from pyAFQ examples. Performs tractography on HARDI dataset.
"""
==========================
Plotting tract profiles
==========================

An example of tracking and segmenting two tracts, and plotting their tract
profiles for FA (calculated with DTI).

"""
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import dipy.data as dpd
from dipy.data import fetcher
import dipy.tracking.utils as dtu
import dipy.tracking.streamline as dts
from dipy.io.streamline import save_tractogram, load_tractogram
from dipy.stats.analysis import afq_profile, gaussian_weights
from dipy.io.stateful_tractogram import StatefulTractogram
from dipy.io.stateful_tractogram import Space

import AFQ.utils.streamlines as aus
import AFQ.data as afd
import AFQ.tractography as aft
import AFQ.registration as reg
import AFQ.dti as dti
import AFQ.segmentation as seg
from AFQ.utils.volume import patch_up_roi
from AFQ.api import make_bundle_dict

def profile(subject):
    bundle_names_afq = ['CST',
                        'UNC',
                        'ARC',
                        'IFO']
    bundles_afq = make_bundle_dict(bundle_names=bundle_names_afq, seg_algo='afq')
    print(f"Fetching HCP subject {subject}")
    afd.fetch_hcp([subject], 
                  profile_name=False,
                  aws_access_key_id=ak,
                  aws_secret_access_key=sk)  
    dwi_dir = op.join(afd.afq_home, 'HCP', 'derivatives',
                      'dmriprep', f'sub-{subject}', 'sess-01/dwi')

    hardi_fdata = op.join(dwi_dir, f"sub-{subject}_dwi.nii.gz")
    hardi_fbval = op.join(dwi_dir, f"sub-{subject}_dwi.bval")
    hardi_fbvec = op.join(dwi_dir, f"sub-{subject}_dwi.bvec")

    img = nib.load(hardi_fdata)

    print("Calculating DTI...")
    if not op.exists(f'./{subject}/dti_FA.nii.gz'):
        dti_params = dti.fit_dti(hardi_fdata, hardi_fbval, hardi_fbvec,
                                 out_dir=f'./{subject}')
    else:
        dti_params = {'FA': f'./{subject}/dti_FA.nii.gz',
                      'params': f'./{subject}/dti_params.nii.gz'}

    FA_img = nib.load(dti_params['FA'])
    FA_data = FA_img.get_fdata()

    templates = afd.read_templates()
    bundles = make_bundle_dict()

    print("Registering to template...")
    MNI_T2_img = dpd.read_mni_template()
    if not op.exists(f'./{subject}/mapping.nii.gz'):
        import dipy.core.gradients as dpg
        gtab = dpg.gradient_table(hardi_fbval, hardi_fbvec)
        warped_hardi, mapping = reg.syn_register_dwi(hardi_fdata, gtab)
        reg.write_mapping(mapping, f'./{subject}/mapping.nii.gz')
    else:
        mapping = reg.read_mapping(f'./{subject}/mapping.nii.gz', img, MNI_T2_img)


    print("Tracking...")
    if not op.exists(f'./{subject}/dti_streamlines.trk'):
        seed_roi = np.zeros(img.shape[:-1])
        for name in bundle_names_afq:
            for hemi in ['_R', '_L']:
                for roi in bundles_afq[name + hemi]['ROIs']:
                    warped_roi = patch_up_roi(
                        (mapping.transform_inverse(
                            roi.get_data().astype(np.float32),
                         interpolation='linear')) > 0)

                    # Add voxels that aren't there yet:
                    seed_roi = np.logical_or(seed_roi, warped_roi)
                
        streamlines = aft.track(dti_params['params'], seed_mask=seed_roi,
                                stop_mask=FA_data, stop_threshold=0.1)

        sft = StatefulTractogram(streamlines, img, Space.RASMM)
        save_tractogram(sft, f'./{subject}/dti_streamlines.trk',
                        bbox_valid_check=False)
    else:
        tg = load_tractogram(f'./{subject}/dti_streamlines.trk', img)
        streamlines = tg.streamlines

    streamlines = dts.Streamlines(
        dtu.transform_tracking_output(streamlines,
                                      np.linalg.inv(img.affine)))

    print("Segmenting...")
    segmentation_reco = seg.Segmentation(algo='reco',
                                         model_clust_thr=20,
                                         reduction_thr=20,
                                         b0_threshold=50,
                                         return_idx=True)
    bundle_names_reco = ['CST',
                         'UF',
                         'AF',
                         'IFOF']
    bundles_reco = make_bundle_dict(bundle_names=bundle_names_reco, seg_algo='reco')
    fiber_groups_reco = segmentation_reco.segment(bundles_reco, streamlines, hardi_fdata, hardi_fbval, hardi_fbvec,
                                                  mapping=mapping, reg_template=MNI_T2_img)
    print(len(fiber_groups_reco))

    segmentation_afq = seg.Segmentation(algo='afq', return_idx=True)
    fiber_groups_afq = segmentation_afq.segment(bundles_afq, streamlines, hardi_fdata, hardi_fbval, hardi_fbvec,
                                                mapping=mapping, reg_template=MNI_T2_img)
    print(len(fiber_groups_afq))

    print("Cleaning...")
    for kk in fiber_groups_reco:
        fiber_groups_reco[kk] = clean_fiber_group(fiber_groups_reco[kk])
    for kk in fiber_groups_afq:
        fiber_groups_afq[kk] = clean_fiber_group(fiber_groups_afq[kk])

    print("Extracting tract profiles...")
    profiles = []
    for kk in fiber_groups_reco:
        weights = gaussian_weights(fiber_groups_reco[kk]['sl'])
        profile = afq_profile(FA_data, fiber_groups_reco[kk]['sl'],
                              np.eye(4), weights=weights)
        for ii in range(len(profile)):
            # Subject, Bundle, node, method, metric (FA, MD), value
            profiles.append([subject, kk, ii, 'reco', 'FA', profile[ii]])

    for kk in fiber_groups_afq:
        weights = gaussian_weights(fiber_groups_afq[kk]['sl'])
        profile = afq_profile(FA_data, fiber_groups_afq[kk]['sl'],
                              np.eye(4), weights=weights)
        for ii in range(len(profile)):
            # Subject, Bundle, node, method, metric (FA, MD), value
            profiles.append([subject, kk, ii, 'afq', 'FA', profile[ii]])

    profiles = pd.DataFrame(data=profiles, columns=["Subject", "Bundle", "Node", "Method", "Metric", "Value"])
    profiles.to_csv(f"./{subject}/profiles.csv")

In [ ]:
subjects = [100408,
            100610,
            101006]
for subject in subjects:
    profile(subject)

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AFQ_data\templates 
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AFQ_data\callosum_templates 
Fetching HCP subject 100408
Calculating DTI...
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AFQ_data\templates 
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AFQ_data\templates 
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AFQ_data\callosum_templates 
Registering to template...
Data size is approximately 70MB
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\.dipy\mni_template 
Tracking...
Segmenting...
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AF

100%|█████████████████████████████████████████████████████████████████████████| 106326/106326 [04:50<00:00, 365.96it/s]


Segmenting...
Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\john\AFQ_data\hcp_atlas_16_bundles 
 Resampled to 20 points
 Size is 28.353 MB
 Duration of resampling is 0.083 sec.
 QBX phase starting...
 Merging phase starting ...
 QuickBundlesX time for 123874 random streamlines
 Duration 2.515 sec. 

 Resampled to 20 points
 Size is 19.869 MB
 Duration of resampling is 0.028 sec.
 QBX phase starting...
 Merging phase starting ...
 QuickBundlesX time for 43405 random streamlines
 Duration 1.594 sec. 

Progressive Registration is Enabled
 Translation  (3 parameters)...
 Rigid  (6 parameters) ...
 Similarity (7 parameters) ...
 Scaling (9 parameters) ...
 Affine (12 parameters) ...
 You have removed all streamlines
 You have removed all streamlines


In [ ]:
profiles = pd.DataFrame(columns=["Subject", "Bundle", "Node", "Method", "Metric", "Value"])
for subject in subjects:
    profiles.append(pd.read_csv(f"./{subject}/profiles.csv"))

profiles_reco = profiles.loc[profiles["Method"] == 'reco', 'Value']
profiles_afq = profiles.loc[profiles["Method"] == 'afq', 'Value']

for i in range(len(profiles_reco)):
    print(numpy.linalg.norm(profiles_reco[i]-profiles_afq[i]))